<a href="https://colab.research.google.com/github/UAlex322/ML-course-master/blob/main/homework_24_10_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Применение методов машинного обучения в задаче классификации с датасетом `monks-problems-1`

####Задача: дан набор векторов $(a_1, a_2, a_3, a_4, a_5, a_6)$, где $a_1 \in \{1, 2, 3\}$, $a_2 \in \{1, 2, 3\}$, $a_3 \in \{1, 2\}$, $a_4 \in \{1, 2, 3\}$, $a_5 \in \{1, 2, 3, 4\}$, $a_6 \in \{1, 2\}$. Требуется обучить модель для проверки истинности логического выражения: $$(a_1 = a_2) \lor (a_5 = 1)$$

In [1]:
import numpy as np
import sklearn
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
%matplotlib inline

**Загрузка датасета:**

In [2]:
X, y = fetch_openml('monks-problems-1', return_X_y=True, as_frame=False)

In [3]:
X

array([['1', '1', '1', '1', '3', '1'],
       ['1', '1', '1', '1', '3', '2'],
       ['1', '1', '1', '3', '2', '1'],
       ...,
       ['3', '3', '2', '3', '3', '2'],
       ['3', '3', '2', '3', '4', '1'],
       ['3', '3', '2', '3', '4', '2']], dtype=object)

In [4]:
X = X.astype('int')
y = y.astype('int')

In [5]:
X

array([[1, 1, 1, 1, 3, 1],
       [1, 1, 1, 1, 3, 2],
       [1, 1, 1, 3, 2, 1],
       ...,
       [3, 3, 2, 3, 3, 2],
       [3, 3, 2, 3, 4, 1],
       [3, 3, 2, 3, 4, 2]])

**Разбиение на тренировочную и тестовую выборки:**

In [6]:
N_total = X.shape[0]
N_train = int(0.8 * N_total)
N_test = N_total - N_train

N_train, N_test

(444, 112)

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = N_train, test_size = N_test, stratify = y, random_state = 19937)

In [8]:
X_train.shape, X_test.shape

((444, 6), (112, 6))

**Применим полносвязную нейронную сеть (MLP):**

In [9]:
from sklearn.neural_network import MLPClassifier

In [10]:
mlp_model = MLPClassifier(hidden_layer_sizes = (20,15), random_state = 19937, max_iter=300)
mlp_model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(20, 15), max_iter=300, random_state=19937)

In [11]:
mlp_y_train_pred = mlp_model.predict(X_train)
mlp_y_test_pred = mlp_model.predict(X_test)
np.mean(y_train != mlp_y_train_pred), np.mean(y_test != mlp_y_test_pred)

(0.0, 0.0)

При указанных параметрах модель абсолютно точно предсказывает ответ на тренировочной и тестовой выборках.

**Применим kNN (k-Nearest Neighbors)**:



In [12]:
from sklearn.neighbors import KNeighborsClassifier

In [13]:
knn_model = KNeighborsClassifier(n_neighbors=10)
knn_model.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=10)

In [14]:
knn_y_train_pred = knn_model.predict(X_train)
knn_y_test_pred = knn_model.predict(X_test)
np.mean(y_train != knn_y_train_pred), np.mean(y_test != knn_y_test_pred)

(0.02702702702702703, 0.017857142857142856)

In [15]:
from sklearn.metrics import confusion_matrix

In [16]:
confusion_matrix(y_train, knn_y_train_pred)

array([[222,   0],
       [ 12, 210]])

In [17]:
confusion_matrix(y_test, knn_y_test_pred)

array([[56,  0],
       [ 2, 54]])

Ошибки на тренировочной и тестовой выборках составляют 2,7% и 1,8% соответственно. Модель ошибается в случаях, когда правильный ответ - `1`, а предсказанный ответ - `0`.

Уменьшим параметр `n_neighbors` до 5:

In [18]:
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)

KNeighborsClassifier()

In [19]:
knn_y_train_pred = knn_model.predict(X_train)
knn_y_test_pred = knn_model.predict(X_test)
np.mean(y_train != knn_y_train_pred), np.mean(y_test != knn_y_test_pred)

(0.015765765765765764, 0.07142857142857142)

На тренировочной выборке результаты стали лучше (1,5% вместо 2,7%), на тестовой - хуже (7,1% вместо 1,8%).

Напротив, увеличим `n_neighbors` до 20:

In [20]:
knn_model = KNeighborsClassifier(n_neighbors=20)
knn_model.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=20)

In [21]:
knn_y_train_pred = knn_model.predict(X_train)
knn_y_test_pred = knn_model.predict(X_test)
np.mean(y_train != knn_y_train_pred), np.mean(y_test != knn_y_test_pred)

(0.06306306306306306, 0.05357142857142857)

Ошибки на тренировочной и тестовой выборках оказались больше, чем при `n_neighbors=10`.

**Подберём оптимальное значение параметра `n_neighbors` на множестве $\{1, 2, \dots, 20\}$:**

In [22]:
from sklearn.model_selection import GridSearchCV

In [23]:
knn_search = GridSearchCV(KNeighborsClassifier(), {'n_neighbors':[i for i in range(1,21)]}, scoring='accuracy', return_train_score=False, verbose=1)

In [24]:
knn_search.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                         13, 14, 15, 16, 17, 18, 19, 20]},
             scoring='accuracy', verbose=1)

Лучшее с точки зрения точности значение параметра - `n_neighbors=9`. Воспользуемся им:

In [25]:
knn_optimal_model = KNeighborsClassifier(n_neighbors=9)
knn_optimal_model.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=9)

In [26]:
knn_opt_y_train_pred = knn_optimal_model.predict(X_train)
knn_opt_y_test_pred = knn_optimal_model.predict(X_test)
np.mean(y_train != knn_opt_y_train_pred), np.mean(y_test != knn_opt_y_test_pred)

(0.01126126126126126, 0.017857142857142856)

Ошибки на тренировочной и тестовой выборках составляют 1,1% и 1,8% соответственно, что лучше, чем при `n_neighbors=10`.